# Vectors

In [1]:
from ipynb.fs.full.koselleck import *
DEFAULT_CORPUS='bpo'

In [2]:
def load_model_row(row): return load_model(row.path)

In [3]:
def get_vec_qstr(row_or_str,run=1):
    if type(run)==int: run=str(run).zfill(2)
    # get key
    if type(row_or_str) in {dict,pd.Series}:
        row=row_or_str
        qstr=f'{row["period"]}_{row["run"]}'
    elif type(row_or_str)==str:
        row_or_str=row_or_str.split('(')[-1].split(')')[0]
        period=row_or_str.split('_')[0]
        run=run if not '_' in row_or_str else str(int(row_or_str.split('_')[1])).zfill(2)
        qstr=f'{period}_{run}'    
    return f'vecs({qstr})'

In [4]:
get_vec_qstr('1750-1755',5)

'vecs(1750-1755_05)'

In [5]:
def vecs1(period,run=1,words=[]):
    qstr=get_vec_qstr(period,run)
    res=None
    with get_veclib('vecs',autocommit=False) as vl:
        if qstr in vl:
            res=vl[qstr]
            if words: res=res.loc[[i for i in res.index if i in set(words)]]
    return res

In [6]:
def vecs(period,run=1,corpus=DEFAULT_CORPUS,words=[]):
    qstr=get_vec_qstr(period,run)
    res=None
    with get_veclib('vecs',autocommit=False) as vl:
        if qstr in vl:
            res=vl[qstr]
        else:
            mpath=os.path.join(PATH_MODELS,corpus,period,f'run_{run:02}','model.bin')
            m=load_model(mpath)
            data=m.wv.vectors
            keys=[m.wv.index_to_key[i] for i in range(len(data))]
            res=pd.DataFrame(data, index=keys)
            vl[qstr]=res
            vl.commit()            
        if words: res=res.loc[[i for i in res.index if i in set(words)]]
    return res

In [7]:
vecs('1700-1740')

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
the,1.322322,-1.788271,0.700118,0.501380,-1.088246,1.377096,-0.763712,-2.323825,0.125689,-1.151854,...,-1.112715,-0.318104,-0.889349,0.664446,-0.533919,-0.509159,1.151490,-0.403696,-0.882792,2.210250
of,0.908900,-1.303617,1.109633,-1.104424,-0.601643,1.056507,2.623128,0.274134,-0.286747,-2.259249,...,0.628232,-1.364491,0.340218,-1.127690,-0.647743,0.513577,0.743693,-0.897379,-1.311412,0.462785
and,0.970605,0.348715,0.964527,0.344566,-0.440003,1.088504,0.279848,0.237195,1.087784,-0.265959,...,1.286765,0.310234,0.495904,-0.194296,-1.595278,0.145395,0.262315,0.438386,-2.042530,0.542283
to,-0.229612,-1.760793,0.540824,0.585672,2.714505,1.172634,-3.903906,-3.897070,3.111161,0.350850,...,-0.617523,-1.604473,-0.123637,-3.046838,0.451828,0.406287,1.835399,-3.388207,-3.655680,-1.623904
in,1.983649,0.465218,2.469530,1.847641,-1.002519,0.423003,-0.573695,-1.896650,0.330863,-0.096130,...,0.453737,0.779045,-0.309466,-0.081862,0.179424,-0.286442,0.673482,-0.063397,-0.916182,2.383888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wetherby,-0.089228,0.083200,0.055142,0.053122,-0.009983,-0.466588,0.044416,0.360859,-0.029012,-0.023452,...,-0.020817,0.065845,0.013593,0.060386,0.009097,0.089561,0.135420,0.112244,0.048280,-0.010008
barbar,-0.123602,0.024903,-0.026472,-0.060492,-0.006487,0.010497,-0.104057,0.257643,0.079990,-0.014209,...,0.051960,0.105872,0.029602,-0.073616,-0.129295,0.053698,0.109924,0.030775,0.007994,-0.020509
wreek,0.042495,0.032689,-0.016970,0.096234,0.054183,-0.265293,0.110970,0.333507,0.014685,0.178063,...,-0.123084,0.034891,-0.023227,0.074813,-0.169850,0.208862,0.228939,-0.137183,0.034377,0.147001
bawn,0.111902,0.009566,-0.099040,-0.004491,0.098914,-0.177088,0.118617,0.089443,0.051087,-0.025362,...,0.000901,-0.062904,0.187166,0.209039,0.221670,0.001659,-0.039167,0.012103,0.237946,0.167782


In [8]:
PATH_MODELS

'/home/ryan/github/koselleck/data1/models'

In [9]:
get_default_models().iloc[0]

corpus                                                                              bpo
period_start                                                                       1720
period_end                                                                         1725
path            /home/ryan/github/koselleck/data1/models/bpo/1720-1725/run_08/model.bin
path_vocab      /home/ryan/github/koselleck/data1/models/bpo/1720-1725/run_08/vocab.txt
run                                                                              run_08
period                                                                        1720-1725
period_len                                                                            5
qstr                                                                 vecs(1720-1725_08)
Name: 533, dtype: object